# Wolverine Glacier Model

## Make a mesh

In [ ]:
import firedrake
from firedrake import inner, grad, dx, Constant, derivative
import icepack
import geojson
import xarray
import geopandas as gpd

In [ ]:
outline_filename = "wolv_tr_dem.geojson"
dataframe = gpd.read_file(outline_filename)
corrected_df = dataframe.to_crs(32606)

corrected_df

In [ ]:
outline_lat_lon = corrected_df.geometry
utm_crs = outline_lat_lon.estimate_utm_crs()

outline_utm = outline_lat_lon.to_crs(utm_crs)
outline_json = geojson.loads(outline_utm.to_json())

outline = geojson.utils.map_tuples(lambda x: x[:2], outline_json)

geometry = icepack.meshing.collection_to_geo(outline)
geo_filename = "wolverine.geo"

with open(geo_filename, "w") as geo_file:
    geo_file.write(geometry.get_code())

In [ ]:
import numpy as np
import icepack.plot

fig, axes = icepack.plot.subplots()

for feature in outline["features"]:
    for line_string in feature["geometry"]["coordinates"]:
        xs = np.array(line_string)
        axes.plot(xs[:, 0], xs[:, 1], linewidth=2)

In [ ]:
!gmsh -2 -format msh2 -v 2 -o wolverine.msh wolverine.geo

In [ ]:
mesh = firedrake.Mesh("wolverine.msh")

In [ ]:
import icepack.plot

fig, axes = icepack.plot.subplots()
firedrake.triplot(mesh, axes=axes)
axes.legend();

In [ ]:
Q = firedrake.FunctionSpace(mesh, "CG", 2)
V = firedrake.VectorFunctionSpace(mesh, "CG", 2)

## Input Data
> Thickness from Farinotti
> 
> Velocity from ITS_LIVE

In [ ]:
!curl -O -C - https://www.research-collection.ethz.ch/bitstream/handle/20.500.11850/315707/results_model_3.zip
!unzip -oj results_model_3.zip RGI60-01/thickness_RGI60-01.09162.tif

In [ ]:
import matplotlib.pyplot as plt
import rasterio

thickness_filename = "thickness_RGI60-01.09162.tif"
thickness_ds = rasterio.open(thickness_filename)

bounds = thickness_ds.bounds
left, right, bottom, top = bounds.left, bounds.right, bounds.bottom, bounds.top
extent = (left, right, bottom, top)

fig, axes = plt.subplots()
axes.imshow(thickness_ds.read(indexes=1), extent=extent)
firedrake.triplot(mesh, interior_kw={"linewidth": 0.02}, axes=axes);

In [ ]:
h_obs = icepack.interpolate(thickness_ds, Q)

In [ ]:
from firedrake import inner, grad, dx
h = h_obs.copy(deepcopy=True)
α = Constant(100.0)
E = 0.5 * ((h - h_obs) ** 2 + α**2 * inner(grad(h), grad(h))) * dx
bcs = firedrake.DirichletBC(Q, h_obs, "on_boundary")
firedrake.solve(firedrake.derivative(E, h) == 0, h, bcs)

In [ ]:
fig, axes = plt.subplots()
axes.set_aspect("equal")
colors = firedrake.tripcolor(h, axes=axes)
fig.colorbar(colors);

In [ ]:
!curl -O -C - https://its-live-data.s3.amazonaws.com/velocity_mosaic/landsat/v00.0/static/cog/ALA_G0120_0000_vx.tif
!curl -O -C - https://its-live-data.s3.amazonaws.com/velocity_mosaic/landsat/v00.0/static/cog/ALA_G0120_0000_vy.tif

In [ ]:
!rio warp ALA_G0120_0000_vx.tif vx.tif --dst-crs EPSG:32606
!rio warp ALA_G0120_0000_vy.tif vy.tif --dst-crs EPSG:32606

In [ ]:
with rasterio.open("vx.tif") as vx_file, rasterio.open("vy.tif") as vy_file:
    u_raw = icepack.interpolate((vx_file, vy_file), V)

We can see from the quiver plot that the directions are rotated by $\pi/2$ clockwise from what they should be.
This is a consequence of the coordinate reprojection that we did from the original polar stereographic projection of the raw data to the UTM projection that we're using here.

In [ ]:
fig, axes = plt.subplots()
axes.set_aspect("equal")
arrows = firedrake.quiver(u_raw, axes=axes)
fig.colorbar(arrows, label="meters/year");

This looks much more sensible.

In [ ]:
u0 = firedrake.interpolate(firedrake.as_vector((-u_raw[1], u_raw[0])), V)

In [ ]:
fig, axes = plt.subplots()
axes.set_aspect("equal")
arrows = firedrake.quiver(u0, axes=axes)
fig.colorbar(arrows, label="meters/year");

Fetch the surface elevation data.
Wolverine is in ArcticDEM tile 48/09.
Then unpack the archive and reproject it to UTM zone 6.

In [ ]:
!curl -O -C - https://data.pgc.umn.edu/elev/dem/setsm/ArcticDEM/mosaic/v4.1/10m/48_09/48_09_10m_v4.1.tar.gz
!tar -xkvf 48_09_10m_v4.1.tar.gz
!rio warp 48_09_10m_v4.1_dem.tif surface.tif --dst-crs EPSG:32606

In [ ]:
with rasterio.open("surface.tif", "r") as arcticdem_file:
    s_obs = icepack.interpolate(arcticdem_file, Q)

s = s_obs.copy(deepcopy=True)
γ = Constant(400.0)
E = 0.5 * ((s - s_obs)**2 + γ**2 * inner(grad(s), grad(s))) * dx
F = firedrake.derivative(E, s)
firedrake.solve(F == 0, s)

In [ ]:
fig, axes = plt.subplots()
axes.set_aspect("equal")
colors = firedrake.tripcolor(s, axes=axes)
firedrake.triplot(mesh, interior_kw={"linewidth": 0.02}, axes=axes)
fig.colorbar(colors);

In [ ]:
from firedrake import grad
grad_s = firedrake.project(-grad(s), V)

fig, axes = plt.subplots()
axes.set_aspect("equal")
colors = firedrake.quiver(grad_s, axes=axes)
fig.colorbar(colors);

Finally, according to the USGS, the annual average temperature is about -1C near the equilibrium line altitude, so we'll use that to set the flow law coefficient in our glacier model.

In [ ]:
A = Constant(icepack.rate_factor(268))

Make the shallow ice velocity.

In [ ]:
ρ_I = Constant(icepack.constants.ice_density)
g = Constant(icepack.constants.gravity)
n = icepack.constants.glen_flow_law

u = firedrake.Function(V)
v = firedrake.TestFunction(V)

P = ρ_I * g * h
S_n = inner(grad(s), grad(s))**((n - 1) / 2)
u_shear = -2 * A * P ** n / (n + 2) * h * S_n * grad(s)
# u_sliding = ...
β = Constant(50.0)
F = (inner(u - u_shear, v) + β**2 * inner(grad(u), grad(v))) * dx

solver_params = {"snes_type": "ksponly", "ksp_type": "gmres"}
fc_params = {"quadrature_degree": 6}
params = {"solver_parameters": solver_params, "form_compiler_parameters": fc_params}
firedrake.solve(F == 0, u, **params)
u_sia = u.copy(deepcopy=True)

In [ ]:
fig, axes = plt.subplots()
axes.set_aspect("equal")
arrows = firedrake.quiver(u, axes=axes)
fig.colorbar(arrows, label="meters/year");